# Events of interest from raw instrumentation logs

In [ ]:
import pandas as pd

logs_df = pd.read_csv('loggedfs_events_pagerank_asymmetrical.log')
cols = [col for col in logs_df.columns if col in ['Action','Time','Path','PID','PPID','UID','Command Line']]
df = logs_df[cols]
df[(df['Action'] == 'read') | (df['Action'] == 'write')]

# Build a Neo4j lineage graph from instrumentation logs

In [ ]:
import lineage_graph as lg

graph = lg.build_lineage_graph('loggedfs_events_pagerank_asymmetrical.log')

# Custom visualization of lineage graph using Vis.js

In [ ]:
from scripts.vis import draw

options = {"Dataset": "name", "Job": "uid"}
draw(graph, options)

# Query Neo4j temporal lineage graph using Cypher

## Query results as a Pandas DataFrame

In [ ]:
# Sanity check query.
query = """
    MATCH (n)-[r]->(m)
    RETURN id(n) AS source_id,
           id(r) AS rel_id,
           id(m) AS target_id
"""

results_df = pd.DataFrame(graph.data(query))
print(results_df)

## Copy detection as a reachability query between 2 content similar datasets

In [ ]:
query = """
    MATCH (n:Dataset)
    WHERE (n.name = '/home/lubuntu/src/file_access_monitor/test_workflows/in1.csv'
    OR n.name = '/home/lubuntu/src/file_access_monitor/test_workflows/out4.csv')
    RETURN id(n) as node_id
"""
node_ids = graph.data(query)

# Path query with monotonically increasing timestamps.
query = """
    START a=node(%d), b=node(%d)
    MATCH p=(a)-[r*]->(b)
    WITH head(relationships(p)) as r1, p, b
    RETURN p, b.name
""" % (node_ids[0]['node_id'], node_ids[1]['node_id'])


paths = graph.data(query)
print('\nFound %d path(s) from a to b:\n' % len(paths))
print(paths)

# Path query with monotonically increasing timestamps.
query = """
    START a=node(%d), b=node(%d)
    MATCH p=(b)-[r*]->(a)
    WITH head(relationships(p)) as r1, p, b
    RETURN p, b.name
""" % (node_ids[0]['node_id'], node_ids[1]['node_id'])


paths = graph.data(query)
print('\nFound %d path(s) from b to a:\n' % len(paths))
print(paths)

## Copy detection as a reachability query between 2 content similar datasets (monotonically increasing edge timestamps)

In [ ]:
query = """
    MATCH (n:Dataset)
    WHERE (n.name = '/home/lubuntu/src/file_access_monitor/test_workflows/in1.csv'
    OR n.name = '/home/lubuntu/src/file_access_monitor/test_workflows/out4.csv')
    RETURN id(n) as node_id
"""
node_ids = graph.data(query)

# Path query with monotonically increasing timestamps.
query = """
    START a=node(%d), b=node(%d)
    MATCH p=(a)-[r*]->(b)
    WITH head(relationships(p)) as r1, p, b
    WHERE all(r2 in relationships(p)
              where r2.timestamp>=r1.timestamp)    
    RETURN p, b.name
""" % (node_ids[0]['node_id'], node_ids[1]['node_id'])


paths = graph.data(query)
print('\nFound %d path(s) from a to b with monotonically increasing timestamps:\n' % len(paths))
print(paths)

# Path query with monotonically increasing timestamps.
query = """
    START a=node(%d), b=node(%d)
    MATCH p=(b)-[r*]->(a)
    WITH head(relationships(p)) as r1, p, b
    WHERE all(r2 in relationships(p)
              where r2.timestamp>=r1.timestamp)    
    RETURN p, b.name
""" % (node_ids[0]['node_id'], node_ids[1]['node_id'])


paths = graph.data(query)
print('\nFound %d path(s) from b to a with monotonically increasing timestamps:\n' % len(paths))
print(paths)

# Aggregate Queries


In [ ]:
query = """
    START from=node(*)
    MATCH p=(from)-->(to)
    WITH from as from, to as to, count(p) as paths
    WHERE paths >= 1
    RETURN to,paths
"""

paths = graph.data(query)
print(paths)

# PageRank

In [ ]:
%load_ext cypher
import networkx as nx
import operator
%matplotlib inline

results = %cypher MATCH p = (a)-[r*]->(b) RETURN p

# Networkx graph.
g = results.get_graph()

#nx.draw(g)
#g.nodes(data=True)

# Print nodes so that we can see their original ids and properties.
print("\nOriginal Nodes:")
print(g.nodes(data=True))

# Node weights for personalized pagerank.
personalize = {}
for node, data in g.nodes(data=True):
    if 'Dataset' in data['labels']:
        personalize[node] = 1
    elif 'Job' in data['labels']:
        # Let's weight each edge by the amount of CPU consumed.
        personalize[node] = data['cpu']
    print("node=%s, data=%s" % (node, data))

print("\nPersonalize Vector:")
print(personalize)

# Transformation from MultiDigraph to Graph for Pagerank calculation.
H = nx.Graph()
for src, dst, edge in g.edges(data=True):
    # Let's weight each edge by the amount of bytes read / written.
    w = edge['data_bytes']
    if H.has_edge(src, dst):
        H[src][dst]['weight'] += w
    else:
        H.add_edge(src, dst, weight=w)

#print("\nPageRank:")
#print(nx.pagerank(H))

print("\nPersonalized PageRank:")
ranks = nx.pagerank(H, personalization=personalize)
sorted_ranks = sorted(ranks.items(), key=operator.itemgetter(1), reverse=True)
print(sorted_ranks)

# Snap Temporal Graph

In [42]:
import time
from py2neo import Graph

# Cleanup all existing graphs.
#query = """
#MATCH (n)
#WITH n limit 100000
#DETACH DELETE n;  
#"""
#graph = Graph()
#graph.data(query)

pwd = os.getcwd()
num_rows = 10000
filename = 'file:///%s/snap_temporal_graph_dataset/email-Eu-core-Dept3-%s.txt' % (pwd, num_rows)

# Load CSV into a neo4j graph.
query = """
USING PERIODIC COMMIT 
LOAD CSV WITH HEADERS FROM '%s'
  AS line
  FIELDTERMINATOR ' '
MERGE (src:Person { id: line.src })
MERGE (dst:Person { id: line.dst })
CREATE (src)-[:SENT_EMAIL_TO {time: line.time}]->(dst)
""" % filename
graph = Graph()
graph.delete_all()

t0 = int(round(time.time() * 1000))
graph.data(query)
t1 = int(round(time.time() * 1000))
print('Time to load %s rows into a neo4j graph: %s ms' % (num_rows, (t1 - t0)))

Time to load 10000 rows into a neo4j graph: 12245 ms


In [44]:
import pandas as pd
import time

query = """
    MATCH (n)-[r]->(m)
    RETURN id(n) AS source_id,
           id(r) AS rel_id,
           id(m) AS target_id
"""

t0 = int(round(time.time() * 1000))
results_df = pd.DataFrame(graph.data(query))
t1 = int(round(time.time() * 1000))

print('Time to fetch temporal graph and make it available as a Pandas dataframe: %s ms' % (t1 - t0))

%load_ext cypher
import networkx as nx
import operator
%matplotlib inline

# Networkx graph.
t0 = int(round(time.time() * 1000))
results = %cypher MATCH p = (a)-[r]->(b) RETURN p
g = results.get_graph()
#nx.draw(g)
g.nodes(data=True)
t1 = int(round(time.time() * 1000))
print('Time to load it into a networkx graph: %s ms' % (t1 - t0))

t0 = int(round(time.time() * 1000))
#print('In-degree: %s' % g.in_degree())
t1 = int(round(time.time() * 1000))
print('Time to compute in-degree: %s ms' % (t1 - t0))

t0 = int(round(time.time() * 1000))       
centrality = nx.betweenness_centrality(g)
#print('Centrality: %s' % centrality)
t1 = int(round(time.time() * 1000))       
print('Time to compute betweeness centrality: %s ms' % (t1 - t0))


# Print nodes so that we can see their original ids and properties.
#print("\nOriginal Nodes:")
#print(g.nodes(data=True))

# Node weights for personalized pagerank.
t0 = int(round(time.time() * 1000))
personalize = {}
for node, data in g.nodes(data=True):
    if 'Person' in data['labels']:
        personalize[node] = 1
    #print("node=%s, data=%s" % (node, data))
t1 = int(round(time.time() * 1000))
print('Time to create PageRank personalization vector: %s ms' % (t1 - t0))

    
#print("\nPersonalize Vector:")
#print(personalize)

# Transformation from MultiDigraph to Graph for Pagerank calculation.
t0 = int(round(time.time() * 1000))
H = nx.Graph()
for src, dst, edge in g.edges(data=True):
    # Let's weight each edge by the amount of bytes read / written.
    w = int(edge['time'])
    if H.has_edge(src, dst):
        H[src][dst]['weight'] += w
    else:
        H.add_edge(src, dst, weight=w)
t1 = int(round(time.time() * 1000))
print('Time to convert from MultiDigraph to Graph for PageRank calculation: %s ms' % (t1 - t0))


t0 = int(round(time.time() * 1000))       
ranks = nx.pagerank(H, personalization=personalize)
sorted_ranks = sorted(ranks.items(), key=operator.itemgetter(1), reverse=True)
t1 = int(round(time.time() * 1000))
print('Time to compute Pagerank: %s ms' % (t1 - t0))

#print("\nPersonalized PageRank:")
#print(sorted_ranks)

Time to fetch temporal graph and make it available as a Pandas dataframe: 660 ms
The cypher extension is already loaded. To reload it, use:
  %reload_ext cypher
10000 rows affected.
Time to load it into a networkx graph: 2281 ms
Time to compute in-degree: 0 ms
Time to compute betweeness centrality: 57 ms
Time to create PageRank personalization vector: 0 ms
Time to convert from MultiDigraph to Graph for PageRank calculation: 37 ms
Time to compute Pagerank: 56 ms
